# **Import used libraries**


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import default_loader
import json
from PIL import Image
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# **Connecting to Google Drive**


Use `flush_and_unmount` and `force_remount=True` for safe connection.

---

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# **Parameters and Folders path**

> Set used parameters' value

In [ ]:
IMAGE_SIZE = 1024  # The size (in pixels) of the images used in the model.
CENTER_CROP_SIZE = 750  # The size (in pixels) for center cropping the images.

NUM_WORKERS_DATALOADER = 6  # The number of data loader workers for parallel data loading.

OUTPUT_FC_UNITS = 2048  # The number of units in the output fully connected layer of the model.

BATCH_SIZE_STEP_ZERO = 32  # Batch size used during the first training step.
BATCH_SIZE_STEP_ONE = 32  # Batch size used during the second training step.
BATCH_SIZE_STEP_EVAL = 32  # Batch size used during model evaluation.

LEARNING_RATE_STEP_ZERO = 0.01  # Learning rate used during the first training step.
LEARNING_RATE_STEP_ONE = 0.001  # Learning rate used during the second training step.

MOMENTUM = 0.9  # Momentum parameter used in the optimization algorithm.

NUM_EPOCHS_STEP_ZERO = 5  # Number of training epochs during the first training step.
NUM_EPOCHS_STEP_ONE = 5  # Number of training epochs during the second training step.

DEFAULT_NUM_CLASS = 3  # Default number of classes for inference; can be changed for different tasks.

> Define used folders path

In [ ]:
DATASET_DIR_PATH = '/content/drive/MyDrive/BSc Project/Skin Cancer Datasets/ISIC_2019_Dataset'
MODEL_CHECKPOINT_PATH = '/content/drive/MyDrive/BSc Project/Codes/Model Checkpoints/Base_Model'
MAPPING_LABEL_INDEX = '/content/drive/MyDrive/BSc Project/Codes/modified_googlenet_lesions_mapping_labels.json'
TRAINING_DATASET_DIR_NAME = 'Trainning Dataset'
VALIDATION_DATASET_DIR_NAME = 'Validation Dataset'
TEST_DATASET_DIR_NAME = 'Test Dataset'

> Find the type of device CPU/GPU

In [ ]:
DEVICE = str(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

# **Implement learning and evaluating functions**

>  Load dataset and create dataloaders for training and validation processes


In [ ]:
def load_data(image_size: int,
              center_crop_size: int,
              dataset_dir_path: str,
              train_dataset_dir_path: str,
              validation_dataset_dir_path: str,
              test_dataset_dir_path: str,
              num_workers_dataloader: int,
              batch_size: int
              ):

  # Define the data transformations (you may need to customize this based on your dataset)
  data_transforms = {
    train_dataset_dir_path : transforms.Compose([
          transforms.Resize(image_size),
          transforms.CenterCrop(center_crop_size),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      ]),
    validation_dataset_dir_path : transforms.Compose([
          transforms.Resize(image_size),
          transforms.CenterCrop(center_crop_size),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      ]),
    test_dataset_dir_path : transforms.Compose([
          transforms.Resize(image_size),
          transforms.CenterCrop(center_crop_size),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      ]),
  }

  # Load the dataset using ImageFolder
  image_datasets = {x: datasets.ImageFolder(os.path.join(dataset_dir_path, x), data_transforms[x]) for x in \
                   [train_dataset_dir_path, validation_dataset_dir_path, test_dataset_dir_path]}

  # Create data loaders
  dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=num_workers_dataloader) for x in \
                [train_dataset_dir_path, validation_dataset_dir_path, test_dataset_dir_path]}

  # Number of classes in your dataset
  num_classes = len(image_datasets['Trainning Dataset'].classes)

  return image_datasets, dataloaders, num_classes

> Implement a function to preprocess input data for inference

In [ ]:
def preprocess_input(image_path):
    transform = transforms.Compose([
        transforms.Resize(1024),
        transforms.CenterCrop(750),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).cuda()  # Add a batch dimension
    return image

> Load model from last checkpoint

In [ ]:
def load_checkpoint(checkpoints_path: str, device):
    try:
      last_checkpoint = sorted(os.listdir(checkpoints_path))[-1]
    except:
      print("There is no saved model in this path: {}".format(checkpoints_path))
      return -1

    checkpoints_path = os.path.join(checkpoints_path, last_checkpoint)

    if device == "cuda:0":
        model = torch.load(checkpoints_path)
        print("checkpoint version {} is loaded on GPU".format(last_checkpoint))

    elif device == "cpu" :
        model = torch.load(checkpoints_path, map_location=torch.device('cpu'))
        print("checkpoint version {} is loaded on CPU".format(last_checkpoint))

    else:
        print("Device Type is unknown")
        return -1

    model.to(device)
    return model

> Save model after training process

In [ ]:
def save_checkpoint(model, checkpoints_path: str, epoch_state: int, step_state: int):
    try:
      last_version = os.listdir(checkpoints_path)[-1].split('_')[-1].split('.')[0]
    except:
      last_version = 0

    model_checkpoint_filename = "Model" + "_" + "Checkpoint" + "_" + "STEP" + "_" + str(step_state) + "EPOCH" + "_" + str(epoch_state) + ".pth"

    save_path = os.path.join(checkpoints_path, model_checkpoint_filename)

    torch.save(model, save_path)
    print("\n checkpoint is saved in path: {}\n".format(save_path))

> Load pre-trained model and append extra layers to the top of pre-trained model

In [ ]:
def load_model(output_fc_units: int, device, checkpoints_path: str, freeze_base_layers: bool, num_classes: int = DEFAULT_NUM_CLASS):

  # check last checkpoint if it exists
  model = load_checkpoint(checkpoints_path, device)

  if model != -1:
    for param in model.parameters():
      param.requires_grad = True
    return model

  # Load a pre-trained GoogleNet model
  model = models.googlenet(pretrained=True)

  # Optionally, you can freeze the parameters of the base model
  if freeze_base_layers:
    for param in model.parameters():
      param.requires_grad = False

  # Modify the final fully connected layer for your number of classes
  num_features = model.fc.in_features
  model.fc = nn.Sequential(
    nn.Linear(num_features, output_fc_units),
    nn.ReLU(),
    nn.Linear(output_fc_units, num_classes),
    nn.Softmax(dim=1),
  )
  # Set the device (GPU or CPU)
  model.to(device)
  print("Model loaded with pure GoogleNet weights")
  return model

> Define learning loss and optimizer

In [ ]:
def define_loss_opt(learning_rate: int, momentum: int):
  # Define loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
  return criterion, optimizer

> Define the training process

In [ ]:
def train(num_epoch: int,
          model,
          train_dataset_dir_path: str,
          validation_dataset_dir_path: str,
          dataloaders,
          image_datasets,
          optimizer,
          criterion,
          device,
          checkpoints_path: str,
          step: int
          ):
  save_checkpoint(model , checkpoints_path, -1, -1)
  # Training loop (you may need to customize this)
  for epoch in range(num_epoch):
      for phase in [train_dataset_dir_path, validation_dataset_dir_path]:
          if phase == train_dataset_dir_path:
              model.train()
          else:
              model.eval()

          running_loss = 0.0
          all_labels = []
          all_preds = []

          for inputs, labels in tqdm(dataloaders[phase], desc="Epoch {} - {} is processing".format(epoch, phase)):
              inputs = inputs.to(device)
              labels = labels.to(device)

              optimizer.zero_grad()

              with torch.set_grad_enabled(phase == train_dataset_dir_path):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  if phase == train_dataset_dir_path:
                      loss.backward()
                      optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              all_labels.extend(labels.cpu().numpy())
              all_preds.extend(preds.cpu().numpy())

          epoch_loss = running_loss / len(image_datasets[phase])
          epoch_acc = accuracy_score(all_labels, all_preds)
          epoch_precision = precision_score(all_labels, all_preds, average='macro')
          epoch_recall = recall_score(all_labels, all_preds, average='macro')
          epoch_f1 = f1_score(all_labels, all_preds, average='macro')

          print(f'{phase} Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.4f} Precision: {epoch_precision:.4f} F1-Score: {epoch_f1:.4f} Recall: {epoch_recall:.4f}')

      # Save the model at the end of each epoch
      save_checkpoint(model , checkpoints_path, epoch, step)

  return model

> Implement model evaluation function

In [ ]:
def evaluation(model, dataloaders, test_dataset_dir_path: str, device):
  # Put the model in evaluation mode
  model.eval()

  # Initialize lists to store true labels and predicted labels
  true_labels = []
  predicted_labels = []
  test_dataloader = dataloaders[test_dataset_dir_path]

  # Iterate through the test dataloader
  with torch.no_grad():  # Disable gradient calculation for evaluation
      for inputs, labels in tqdm(test_dataloader, desc="Evaluating in progress... "):
          inputs = inputs.to(device)  # Move data to the device (e.g., GPU)
          labels = labels.to(device)

          # Forward pass to get model predictions
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)  # Get predicted class indices

          # Append true labels and predicted labels to lists
          true_labels.extend(labels.cpu().numpy())
          predicted_labels.extend(preds.cpu().numpy())

  # Calculate evaluation metrics

  accuracy = accuracy_score(true_labels, predicted_labels)
  precision = precision_score(true_labels, predicted_labels, average='macro')
  recall = recall_score(true_labels, predicted_labels, average='macro')
  f1 = f1_score(true_labels, predicted_labels, average='macro')
  conf_matrix = confusion_matrix(true_labels, predicted_labels)

  print(f'\nAccuracy: {accuracy:.4f}')
  print(f'Precision: {precision:.4f}')
  print(f'Recall: {recall:.4f}')
  print(f'F1-Score: {f1:.4f}')
  print('Confusion Matrix:')
  print(conf_matrix)

  return accuracy, precision, recall, f1, conf_matrix

> Function to get predicted label

In [ ]:
def get_predicted_label(output_tensor, class_idx):
    _, predicted_idx = output_tensor.max(1)
    predicted_label = class_idx[str(predicted_idx.item())][1]
    return predicted_label

> Prediction for a dermoscopic image

In [ ]:
def prediction(image_path: str, mapping_label_index: str):
  # Set the model to evaluation mode
  model.eval()

  # Load the class index mapping from the JSON file
  with open(mapping_label_index) as f:
    class_idx = json.load(f)

  input_image = preprocess_input(image_path)

  # Perform inference
  with torch.no_grad():
    output = model(input_image)

  # get predicted label
  predicted_label = get_predicted_label(output, class_idx)
  print(f"Predicted Label: {predicted_label}")

  return predicted_label

# **Learning Model Process**



## ***Phase 0: Train only added extra layers***


> Load datasets and create handler for them and # classes of data

In [ ]:
image_datasets, dataloaders, num_classes = load_data(image_size=IMAGE_SIZE,
                                                     center_crop_size=CENTER_CROP_SIZE,
                                                     dataset_dir_path=DATASET_DIR_PATH,
                                                     train_dataset_dir_path=TRAINING_DATASET_DIR_NAME,
                                                     validation_dataset_dir_path=VALIDATION_DATASET_DIR_NAME,
                                                     test_dataset_dir_path=TEST_DATASET_DIR_NAME,
                                                     num_workers_dataloader=NUM_WORKERS_DATALOADER,
                                                     batch_size=BATCH_SIZE_STEP_ZERO)

> Load only pre-trained model or load from checkpoint

In [ ]:
model = load_model(num_classes=num_classes, output_fc_units=OUTPUT_FC_UNITS, device=DEVICE, checkpoints_path=MODEL_CHECKPOINT_PATH, freeze_base_layers=True)

> Create learning loss and optimizer

In [ ]:
criterion, optimizer = define_loss_opt(learning_rate=LEARNING_RATE_STEP_ZERO, momentum=MOMENTUM)

> Train model for the first phase

In [ ]:
model = train(num_epoch=NUM_EPOCHS_STEP_ZERO,
          model=model,
          train_dataset_dir_path=TRAINING_DATASET_DIR_NAME,
          validation_dataset_dir_path=VALIDATION_DATASET_DIR_NAME,
          dataloaders=dataloaders,
          image_datasets=image_datasets,
          optimizer=optimizer,
          criterion=criterion,
          device=DEVICE,
          checkpoints_path=MODEL_CHECKPOINT_PATH,
          step=0)

## ***Phase 1: Train all layers***


In [ ]:
image_datasets, dataloaders, num_classes = load_data(image_size=IMAGE_SIZE,
                                                     center_crop_size=CENTER_CROP_SIZE,
                                                     dataset_dir_path=DATASET_DIR_PATH,
                                                     train_dataset_dir_path=TRAINING_DATASET_DIR_NAME,
                                                     validation_dataset_dir_path=VALIDATION_DATASET_DIR_NAME,
                                                     test_dataset_dir_path=TEST_DATASET_DIR_NAME,
                                                     num_workers_dataloader=NUM_WORKERS_DATALOADER,
                                                     batch_size=BATCH_SIZE_STEP_ONE)

In [ ]:
model = load_model(num_classes=num_classes, output_fc_units=OUTPUT_FC_UNITS, device=DEVICE, checkpoints_path=MODEL_CHECKPOINT_PATH, freeze_base_layers=False)

In [ ]:
criterion, optimizer = define_loss_opt(learning_rate=LEARNING_RATE_STEP_ONE, momentum=MOMENTUM)

In [ ]:
model = train(num_epoch=NUM_EPOCHS_STEP_ONE,
          model=model,
          train_dataset_dir_path=TRAINING_DATASET_DIR_NAME,
          validation_dataset_dir_path=VALIDATION_DATASET_DIR_NAME,
          dataloaders=dataloaders,
          image_datasets=image_datasets,
          optimizer=optimizer,
          criterion=criterion,
          device=DEVICE,
          checkpoints_path=MODEL_CHECKPOINT_PATH,
          step=1)

# **Evaluate Model Process**

> Load dataset for model evaluation

In [ ]:
image_datasets, dataloaders, num_classes = load_data(image_size=IMAGE_SIZE,
                                                     center_crop_size=CENTER_CROP_SIZE,
                                                     dataset_dir_path=DATASET_DIR_PATH,
                                                     train_dataset_dir_path=TRAINING_DATASET_DIR_NAME,
                                                     validation_dataset_dir_path=VALIDATION_DATASET_DIR_NAME,
                                                     test_dataset_dir_path=TEST_DATASET_DIR_NAME,
                                                     num_workers_dataloader=NUM_WORKERS_DATALOADER,
                                                     batch_size=BATCH_SIZE_STEP_EVAL)

In [ ]:
model = load_model(num_classes=num_classes, output_fc_units=OUTPUT_FC_UNITS, device=DEVICE, checkpoints_path=MODEL_CHECKPOINT_PATH, freeze_base_layers=True)



> Evaluate model and show and return learning metrics



In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluation(model=model, dataloaders=dataloaders, test_dataset_dir_path=TEST_DATASET_DIR_NAME, device=DEVICE)

# **Inference from the loaded model**

In [ ]:
model = load_model(output_fc_units=OUTPUT_FC_UNITS, device=DEVICE, checkpoints_path=MODEL_CHECKPOINT_PATH, freeze_base_layers=True)

> Predict a skin cancer image

In [ ]:
image_path = "/content/drive/MyDrive/BSc Project/Skin Cancer Datasets/ISIC_2019_Dataset/Validation Dataset/melanoma/ISIC_0026115.jpg"
prediction(image_path=image_path, mapping_label_index=MAPPING_LABEL_INDEX)